In [1]:
using JuMP
using MathProgBase

m = Model()

@variable(m, 3.0 <= var[1:2] <= 5.0)
@NLobjective(m, Min, 123.45*(var[1] - pi)^2 + sin(var[1]) + (var[2] - e)^2 + sin(pi - var[2]))

x = Vector{Float64}(2)
x[linearindex(var[1])] = 3.5
x[linearindex(var[2])] = 2.0
d = JuMP.NLPEvaluator(m)
MathProgBase.initialize(d, [:Grad, :Hess])
objval = MathProgBase.eval_f(d, x) 
println("The objective at ", x, " is ", objval, ".")
g = Vector{Float64}(2)
MathProgBase.eval_grad_f(d, g, x)
g


The objective at [3.5, 2.0] is 16.932314699092036.


2-element Array{Float64,1}:
 87.5543 
 -1.85271

In [7]:
function my_f(n::Cint, x_ptr::Ptr{Float64}, obj_ptr::Ptr{Float64}, flag_ptr::Ptr{Cint})
    x = unsafe_wrap(Array, x_ptr, Int(n))
    obj_val = MathProgBase.eval_f(d, x)
    unsafe_store!(obj_ptr, obj_val)
    unsafe_store!(flag_ptr, 0)
    nothing
end

const my_f_c = cfunction(my_f, Void, (Cint, Ptr{Float64}, Ptr{Float64}, Ptr{Cint}))

function my_evalg(n::Cint, x_ptr::Ptr{Float64}, g_ptr::Ptr{Float64}, flag_ptr::Ptr{Cint})
    x = unsafe_wrap(Array, x_ptr, Int(n))
    g = unsafe_wrap(Array, g_ptr, Int(n))
    MathProgBase.eval_grad_f(d, g, x)

    unsafe_store!(flag_ptr, 0)
    nothing
end

const my_evalg_c = cfunction(my_evalg, Void, (Cint, Ptr{Float64}, Ptr{Float64}, Ptr{Cint}))

function my_evalh(n::Cint, x_ptr::Ptr{Float64}, 
                  hrow_ptr::Ptr{Cint}, hcol_ptr::Ptr{Cint}, hval_ptr::Ptr{Float64}, hnnz_ptr::Ptr{Cint},
                  lim::Cint, lmem_ptr::Ptr{UInt8}, flag_ptr::Ptr{Cint})
    # TODO: This should be moved out, and called only once
    I, J = MathProgBase.hesslag_structure(d)
    nnz = length(I)
    if nnz > Int(lim)
        unsafe_store!(lmem_ptr, 1)
        unsafe_store!(flag_ptr, 1)
        return nothing
    end
    unsafe_store!(hnnz_ptr, nnz)
    hcol_ind = unsafe_wrap(Array, hcol_ptr, Int(lim))
    hrow_ind = unsafe_wrap(Array, hrow_ptr, Int(lim))
    
    # TODO: This is really bad and only happens because I am calling C instead of Fotran.
    # TODO: Try to move to direct fortran calling.
    hcol_ind[1:nnz] = J - 1
    hrow_ind[1:nnz] = I - 1

    σ = 1.0
    μ = Vector{Float64}(0)
    x = unsafe_wrap(Array, x_ptr, Int(n))
    H = unsafe_wrap(Array, hval_ptr, Int(lim))
    MathProgBase.eval_hesslag(d, H, x, σ, μ)
    
    unsafe_store!(flag_ptr, 0)
    
    nothing
end

const my_evalh_c = cfunction(my_evalh, Void,
    (Cint, Ptr{Float64}, Ptr{Cint}, Ptr{Cint}, Ptr{Float64}, Ptr{Cint}, Cint, Ptr{UInt8}, Ptr{Cint}))

# Problems functions 
myevalf = my_f_c
myevalg = my_evalg_c
myevalh = my_evalh_c
myevalc = C_NULL
myevaljac = C_NULL
myevalhc = C_NULL
myevalfc = C_NULL
myevalgjac = C_NULL
myevalgjacp = C_NULL
myevalhl = C_NULL
myevalhlp = C_NULL
jcnnzmax = 0
hnnzmax = 2
# Parameters setting
epsfeas = [1.0e-08]
epsopt  = [1.0e-08]
efstin  = sqrt.(epsfeas)
eostin  = epsopt.^1.5
efacc   = sqrt.(epsfeas)
eoacc   = sqrt.(epsopt)

outputfnm = "algencan.out"
specfnm   = ""

vparam = ["ITERATIONS-OUTPUT-DETAIL 1"] #Vector{String}(nvparam)
nvparam = length(vparam)

n = 2
l = 3*ones(n)
u = 5*ones(n)

# Information on the constraints that do not exist
m = 0
lambda = Vector{Float64}(m)
equatn = Vector{UInt8}(m)
linear = Vector{UInt8}(m)

coded = zeros(UInt8, 11)
coded[1:3] = 1
checkder = UInt8(0)
f = [0.0]
cnorm = [0.0]
snorm = [0.0]
nlpsupn = [0.0]

inform = Vector{Cint}([0])

b = ccall(
    (:c_algencan, "algencan-3.1.1/lib/libalgencan.so"), # library
    Void,                                               # Return type
    (                                    # Parameters types
        Ptr{Void},                                   # *myevalf, 
        Ptr{Void},                                   # *myevalg, 
        Ptr{Void},                                   # *myevalh, 
        Ptr{Void},                                   # *myevalc,
        Ptr{Void},                                   # *myevaljac, 
        Ptr{Void},                                   # *myevalhc, 
        Ptr{Void},                                   # *myevalfc,
        Ptr{Void},                                   # *myevalgjac,
        Ptr{Void},                                   # *myevalgjacp, 
        Ptr{Void},                                   # *myevalhl, 
        Ptr{Void},                                   # *myevalhlp, 
        Cint,                                        # jcnnzmax,
        Cint,                                        # hnnzmax,
        Ref{Cdouble},                                # *epsfeas, 
        Ref{Cdouble},                                # *epsopt, 
        Ref{Cdouble},                                # *efstin,
        Ref{Cdouble},                                # *eostin, 
        Ref{Cdouble},                                # *efacc, 
        Ref{Cdouble},                                # *eoacc, 
        Cstring,                                     # *outputfnm,
        Cstring,                                     # *specfnm, 
        Cint,                                        # nvparam,
        Ptr{Ptr{UInt8}},                             # **vparam, 
        Cint,                                        # int n, 
        Ref{Cdouble},                                # *x,
        Ref{Cdouble},                                # *l, 
        Ref{Cdouble},                                # *u, 
        Cint,                                        #  m, 
        Ref{Cdouble},                                # double *lambda, 
        Ref{UInt8},                                  # *equatn,
        Ref{UInt8},                                  # _Bool *linear, 
        Ref{UInt8},                                  # _Bool *coded, 
        UInt8,                                       # _Bool checkder, 
        Ref{Cdouble},                                # double *f,
        Ref{Cdouble},                                # double *cnorm, 
        Ref{Cdouble},                                # double *snorm, 
        Ref{Cdouble},                                # double *nlpsupn,
        Ref{Cint}                                    # int *inform
    ),
    myevalf, 
    myevalg, 
    myevalh, 
    myevalc,
    myevaljac, 
    myevalhc, 
    myevalfc,
    myevalgjac,
    myevalgjacp, 
    myevalhl, 
    myevalhlp, 
    jcnnzmax,
    hnnzmax,
    epsfeas, 
    epsopt, 
    efstin,
    eostin, 
    efacc, 
    eoacc, 
    outputfnm,
    specfnm, 
    nvparam,
    vparam, 
    n, 
    x,
    l, 
    u, 
    m, 
    lambda, 
    equatn,
    linear, 
    coded, 
    checkder, 
    f,
    cnorm, 
    snorm, 
    nlpsupn,
    inform    
)
println("x final ", x)


 This is ALGENCAN 3.1.1.
 ALGENCAN, an Augmented Lagrangian method for nonlinear programming, is part of
 the TANGO Project: Trustable Algorithms for Nonlinear General Optimization.
 See http://www.ime.usp.br/~egbirgin/tango/ for details.

 Processing array of parameters with   1 entrances:

 ITERATIONS-OUTPUT-DETAIL                                                     1

 The specification file is not being used.

 Available HSL subroutines = NONE           

 ALGENCAN PARAMETERS:

 firstde                =                    T
 seconde                =                    T
 truehpr                =                    T
 hptype in TN           =               TRUEHP
 lsslvr in TR           =            NONE/NONE
 lsslvr in NW           =            NONE/NONE
 lsslvr in ACCPROC      =            NONE/NONE
 innslvr                =                   TN
 accproc                =                    F
 rmfixv                 =                    T
 slacks                 =                 